# Setting Up

In [1]:
!nvidia-smi

Sun Jan 21 16:15:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [10]:
! pip install -q transformers[sentencepiece] fastbook fastai ohmeow-blurr nbdev

In [7]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [9]:
! pip install -q onnxruntime onnx onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 MB 5.9 MB/s eta 0:00:00


In [11]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [12]:
from tqdm.notebook import tqdm
import numpy as np

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
%cd /content/drive/MyDrive/MasterCourse/LanguageModeling

/content/drive/MyDrive/MasterCourse/LanguageModeling


In [15]:
df = pd.read_csv("/content/drive/MyDrive/MasterCourse/LanguageModeling/anime_genre_details_merged.csv")
df.head()

,title_name,title_english,url,score_label,description,episodes,aired,premiered,genres
0,Sousou no Frieren,Frieren: Beyond Journey's End,https://myanimelist.net/anime/52991/Sousou_no_Frieren,9.14,"During their decade-long quest to defeat the Demon King, the members of the hero's party—Himmel himself, the priest Heiter, the dwarf warrior Eisen, and the elven mage Frieren—forge bonds through adventures and battles, creating unforgettable precious memories for most of them.\n\nHowever, the time that Frieren spends with her comrades is equivalent to merely a fraction of her life, which has lasted over a thousand years. When the party disbands after their victory, Frieren casually returns to her ""usual"" routine of collecting spells across the continent. Due to her different sense of time...",28,"Sep 29, 2023 to ?",Fall 2023,"['Adventure', 'Drama', 'Fantasy', 'Shounen']"
1,Fullmetal Alchemist: Brotherhood,NaN,https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood,9.09,"After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.\n\nThe brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-...",64,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,"['Action', 'Adventure', 'Drama', 'Fantasy', 'Military', 'Shounen']"
2,Steins;Gate,NaN,https://myanimelist.net/anime/9253/Steins_Gate,9.07,"Eccentric scientist Rintarou Okabe has a never-ending thirst for scientific exploration. Together with his ditzy but well-meaning friend Mayuri Shiina and his roommate Itaru Hashida, Rintarou founds the Future Gadget Laboratory in the hopes of creating technological innovations that baffle the human psyche. Despite claims of grandeur, the only notable ""gadget"" the trio have created is a microwave that has the mystifying power to turn bananas into green goo.\n\nHowever, when Rintarou decides to attend neuroscientist Kurisu Makise's conference on time travel, he experiences a series of stran...",24,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,"['Drama', 'Sci-Fi', 'Suspense', 'Psychological', 'Time Travel']"
3,Gintama°,Gintama Season 4,https://myanimelist.net/anime/28977/Gintama°,9.06,"Gintoki, Shinpachi, and Kagura return as the fun-loving but broke members of the Yorozuya team! Living in an alternate-reality Edo, where swords are prohibited and alien overlords have conquered Japan, they try to thrive on doing whatever work they can get their hands on. However, Shinpachi and Kagura still haven't been paid... Does Gin-chan really spend all that cash playing pachinko?\n\nMeanwhile, when Gintoki drunkenly staggers home one night, an alien spaceship crashes nearby. A fatally injured crew member emerges from the ship and gives Gintoki a strange, clock-shaped device, warning ...",51,"Apr 8, 2015 to Mar 30, 2016",Spring 2015,"['Action', 'Comedy', 'Sci-Fi', 'Gag Humor', 'Historical', 'Parody', 'Samurai', 'Shounen']"
4,Shingeki no Kyojin Season 3 Part 2,Attack on Titan Season 3 Part 2,https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2,9.05,"Seeking to restore humanity's diminishing hope, the Survey Corps embark on a mission to retake Wall Maria, where the battle against the merciless ""Titans"" takes the stage once again.\n\nReturning to the tattered Shiganshina District that was once his home, Eren Yeager and the Corps find the town oddly unoccupied by Titans. Even after the outer gate is plugged, they strangely encounter no opposition. The mission progresses smoothly until Armin Arlert, highly suspicious of the enemy's absence, discovers distressing signs of a potential sche

In [16]:
df.drop(["title_english","premiered"],axis = 1, inplace = True)
df.head(5)

,title_name,url,score_label,description,episodes,aired,genres
0,Sousou no Frieren,https://myanimelist.net/anime/52991/Sousou_no_Frieren,9.14,"During their decade-long quest to defeat the Demon King, the members of the hero's party—Himmel himself, the priest Heiter, the dwarf warrior Eisen, and the elven mage Frieren—forge bonds through adventures and battles, creating unforgettable precious memories for most of them.\n\nHowever, the time that Frieren spends with her comrades is equivalent to merely a fraction of her life, which has lasted over a thousand years. When the party disbands after their victory, Frieren casually returns to her ""usual"" routine of collecting spells across the continent. Due to her different sense of time...",28,"Sep 29, 2023 to ?","['Adventure', 'Drama', 'Fantasy', 'Shounen']"
1,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood,9.09,"After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.\n\nThe brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-...",64,"Apr 5, 2009 to Jul 4, 2010","['Action', 'Adventure', 'Drama', 'Fantasy', 'Military', 'Shounen']"
2,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,9.07,"Eccentric scientist Rintarou Okabe has a never-ending thirst for scientific exploration. Together with his ditzy but well-meaning friend Mayuri Shiina and his roommate Itaru Hashida, Rintarou founds the Future Gadget Laboratory in the hopes of creating technological innovations that baffle the human psyche. Despite claims of grandeur, the only notable ""gadget"" the trio have created is a microwave that has the mystifying power to turn bananas into green goo.\n\nHowever, when Rintarou decides to attend neuroscientist Kurisu Makise's conference on time travel, he experiences a series of stran...",24,"Apr 6, 2011 to Sep 14, 2011","['Drama', 'Sci-Fi', 'Suspense', 'Psychological', 'Time Travel']"
3,Gintama°,https://myanimelist.net/anime/28977/Gintama°,9.06,"Gintoki, Shinpachi, and Kagura return as the fun-loving but broke members of the Yorozuya team! Living in an alternate-reality Edo, where swords are prohibited and alien overlords have conquered Japan, they try to thrive on doing whatever work they can get their hands on. However, Shinpachi and Kagura still haven't been paid... Does Gin-chan really spend all that cash playing pachinko?\n\nMeanwhile, when Gintoki drunkenly staggers home one night, an alien spaceship crashes nearby. A fatally injured crew member emerges from the ship and gives Gintoki a strange, clock-shaped device, warning ...",51,"Apr 8, 2015 to Mar 30, 2016","['Action', 'Comedy', 'Sci-Fi', 'Gag Humor', 'Historical', 'Parody', 'Samurai', 'Shounen']"
4,Shingeki no Kyojin Season 3 Part 2,https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2,9.05,"Seeking to restore humanity's diminishing hope, the Survey Corps embark on a mission to retake Wall Maria, where the battle against the merciless ""Titans"" takes the stage once again.\n\nReturning to the tattered Shiganshina District that was once his home, Eren Yeager and the Corps find the town oddly unoccupied by Titans. Even after the outer gate is plugged, they strangely encounter no opposition. The mission progresses smoothly until Armin Arlert, highly suspicious of the enemy's absence, discovers distressing signs of a potential scheme against them. \n\nShingeki no Kyojin Season 3 Par...",10,"Apr 29, 2019 to Jul 1, 2019","['Action', 'Drama', 'Suspense', 'Gore', 'Military', 'Survival', 'S

In [17]:
df = df.dropna().reset_index(drop=True)
df.isna().sum()

title_name     0
url            0
score_label    0
description    0
episodes       0
aired          0
genres         0
dtype: int64

In [19]:
df.shape

(8947, 7)

In [23]:
genres_list = df.genres.to_list()
genre_count = {}
for genres in genres_list:
  genre_list = eval(genres)
  for genre in genre_list:
    if genre in genre_count.keys():
      genre_count[genre] += 1
    else:
      genre_count[genre] = 1
print(f"Number of Genres: {len(genre_count)}")
print(genre_count)

threshold = int(len(df) * 0.01)
rare_genres = [key for key, value in genre_count.items() if value < threshold]
len(rare_genres)

Number of Genres: 74
{'Adventure': 2194, 'Drama': 1774, 'Fantasy': 2593, 'Shounen': 1599, 'Action': 3234, 'Military': 490, 'Sci-Fi': 1836, 'Suspense': 173, 'Psychological': 265, 'Time Travel': 114, 'Comedy': 3594, 'Gag Humor': 181, 'Historical': 729, 'Parody': 313, 'Samurai': 129, 'Gore': 195, 'Survival': 75, 'Adult Cast': 497, 'Space': 351, 'Romance': 1453, 'School': 1431, 'Seinen': 761, 'Supernatural': 898, 'Shoujo': 348, 'Award Winning': 189, 'Romantic Subtext': 45, 'Childcare': 51, 'Strategy Game': 153, 'Mecha': 756, 'Super Power': 455, 'Mystery': 686, 'Vampire': 126, 'Medical': 34, 'Sports': 441, 'Team Sports': 200, 'CGDCT': 209, 'Music': 394, 'Combat Sports': 65, 'Mythology': 385, 'Slice of Life': 667, 'Iyashikei': 148, 'Love Polygon': 77, 'Performing Arts': 79, 'Josei': 109, 'Reincarnation': 94, 'Showbiz': 30, 'Delinquents': 38, 'Workplace': 129, 'Ecchi': 570, 'Isekai': 264, 'Anthropomorphic': 171, 'Organized Crime': 59, 'Detective': 240, 'Horror': 251, 'Avant Garde': 47, 'Mahou

24

In [24]:
genres_list = df.genres.to_list()
revised_genre_list = []
indices_to_drop = []

for idx, genres in enumerate(genres_list):
  genre_list = eval(genres)
  revised_genres = []

  for genre in genre_list:
    if genre not in rare_genres:
      revised_genres.append(genre)

  if len(revised_genres) == 0:
    indices_to_drop.append(idx)
  else:
    revised_genre_list.append(revised_genres)

df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_genres'] = revised_genre_list
df.shape

(8927, 8)

In [25]:
revised_genres_list = df.revised_genres.to_list()
revised_genre_count = {}
for genres in revised_genres_list:
  genre_list = genres
  for genre in genre_list:
    if genre in revised_genre_count.keys():
      revised_genre_count[genre] += 1
    else:
      revised_genre_count[genre] = 1
print(f"Number of Genres: {len(revised_genre_count)}")

Number of Genres: 50


In [26]:
encode_genre_types = { key: idx for idx, (key, value) in enumerate(revised_genre_count.items())}
with open("genre_types_encoded.json", "w") as fp:
  json.dump(encode_genre_types, fp)

# We need this because for multilabel classification all genres have possibility to be present in the predictions
categorical_genre_list = []
revised_genres_list = df.revised_genres.to_list()

for revised_genres in revised_genres_list:
  categorical_list = [0] * len(encode_genre_types)
  for genre in revised_genres:
    genre_type_index = encode_genre_types[genre]
    categorical_list[genre_type_index] = 1
  categorical_genre_list.append(categorical_list)

df['genre_cat_list'] = categorical_genre_list
df.shape

(8927, 9)

In [27]:
labels = list(encode_genre_types.keys())
len(labels), labels[:5]

(50, ['Adventure', 'Drama', 'Fantasy', 'Shounen', 'Action'])

In [28]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(8035, 892)

In [29]:
valid_df = df.loc[valid_ids]
valid_df.head()

,title_name,url,score_label,description,episodes,aired,genres,revised_genres,genre_cat_list
1069,BanG Dream! Garupa☆Pico: Oomori,https://myanimelist.net/anime/40854/BanG_Dream_Garupa☆Pico__Oomori,7.74,The second season of the mini series starring the girls of BanG Dream!,26,"May 7, 2020 to Oct 29, 2020","['Comedy', 'Music']","[Comedy, Music]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4492,Blade Runner: Black Out 2022,https://myanimelist.net/anime/36308/Blade_Runner__Black_Out_2022,6.91,"In 2022, an EMP detonation causes a global blackout that has massive, destructive implications all over the world. Directed by Cowboy Bebop and Samurai Champloo's Shinichiro Watanabe, Blade Runner: Black Out 2022 is an animated short which serves as a prologue for the feature film Blade Runner 2049.\n\n(Source: Crunchyroll, edited)",1,26-Sep-17,"['Sci-Fi', 'Suspense']","[Sci-Fi, Suspense]","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1203,Shigatsu wa Kimi no Uso: Moments,https://myanimelist.net/anime/28069/Shigatsu_wa_Kimi_no_Uso__Moments,7.69,"OVA bundled with the 11th Shigatsu wa Kimi no Uso manga volume.\n\nThe OVA will focus on Kousei, Emi, and Takeshi when they were younger, showing their troubles during their competitions and the beginning of their rivalry.\n\n(Source: Shigatsu wa Kimi no Uso Wikia)",1,15-May-15,"['Music', 'Shounen']","[Music, Shounen]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
8925,Manga Nippon Mukashibanashi,https://myanimelist.net/anime/10282/Manga_Nippon_Mukashibanashi,6.05,This series is an anthology of old Japanese tales.\n\n(Source: AniDB),12,"Jan 7, 1975 to Mar 25, 1975",['Historical'],[Historical],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4825,Kaibutsu-kun,https://myanimelist.net/anime/3129/Kaibutsu-kun,6.84,"The prince of Kaibutsu Land, Kaibutsu-kun, decides to go to the human world as part of his training to become the King.\n\n(Source: ANN)",49,"Apr 21, 1968 to Mar 23, 1969","['Comedy', 'Horror']","[Comedy, Horror]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


#Fastai & Blurr Inference

In [31]:
pwd

'/content/drive/MyDrive/MasterCourse/LanguageModeling'

In [32]:
model_path = "/content/drive/MyDrive/MasterCourse/LanguageModeling/models (1)/anime-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [33]:
learner_inf.blurr_predict("Hero reborns")

[{'labels': ['Action'],
  'scores': [0.733512282371521],
  'class_indices': [0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'class_labels': ['Adventure', 'Drama', 'Fantasy', 'Shounen', 'Action', 'Military', 'Sci-Fi', 'Suspense', 'Psychological', 'Time Travel', 'Comedy', 'Gag Humor', 'Historical', 'Parody', 'Samurai', 'Gore', 'Adult Cast', 'Space', 'Romance', 'School', 'Seinen', 'Supernatural', 'Shoujo', 'Award Winning', 'Strategy Game', 'Mecha', 'Super Power', 'Mystery', 'Vampire', 'Sports', 'Team Sports', 'CGDCT', 'Music', 'Mythology', 'Slice of Life', 'Iyashikei', 'Josei', 'Reincarnation', 'Workplace', 'Ecchi', 'Isekai', 'Anthropomorphic', 'Detective', 'Horror', 'Mahou Shoujo', 'Martial Arts', 'Kids', 'Idols (Female)', 'Video Game', 'H

In [34]:
learner_inf.blurr_predict("Hero reborns")[0]['labels']

['Action']

#Evaluation

In [35]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['genre_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [36]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['description']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

preds[0][:20]

  0%|          | 0/892 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [37]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.44868637110016424
F1 Score (Macro) = 0.17185730042226727


#Convert to ONNX

In [40]:
model_path = "/content/drive/MyDrive/MasterCourse/LanguageModeling/models (1)/anime-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [42]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'onnx/anime-classifier.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [43]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = 'onnx/anime-classifier.onnx'
quantized_onnx_model_path = 'onnx/anime-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

## Normal ONNX

In [44]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('onnx/anime-classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [45]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/892 [00:00<?, ?it/s]

In [46]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.4470491466173144
F1 Score (Macro) = 0.1713665500515735


## Quantized ONNX

In [47]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('onnx/anime-classifier-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [48]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/892 [00:00<?, ?it/s]

In [49]:
metric_measures(valid_df, preds) #

F1 Score (Micro) = 0.44705397610218367
F1 Score (Macro) = 0.17351732264303135
